https://www.youtube.com/watch?v=YHBNCESKyrM

# Convolutional Neural Network

- MNIST data
- 3 convolutional layers
- 2 fully connected layers

## 1. Settings

### 1) Import required libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import time

### 2) Set hyperparameters

In [40]:
batch_size = 128
learning_rate = 0.0002
num_epoch = 20


is_gpu = True
#is_gpu = False

## 2. Data

### 1) Download data

In [41]:
mnist_train = dset.MNIST('./', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST('./', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

### 2) Check data

In [42]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [43]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

## 3. Model & Optimizer

### 1) CNN Model

In [44]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,64,3,1,1),    # 64 * 28 * 28
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64,128,3,1,1),  # 128 * 28 * 28
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2),        # 128 * 14 * 14
            nn.Conv2d(128,256,3,1,1), # 256 * 14 * 14
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256,512,3,1,1), # 512 * 14 * 14
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2,2)        # 512 * 7 * 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(512*7*7, 400),
            nn.ReLU(),
            nn.Linear(400,10)
        )
        
    def forward(self, x):
        out = self.layer(x)
        #out = out.view(batch_size, -1)
        out = out.view(-1, 512*7*7)
        out = self.fc_layer(out)
        return out
    
model = CNN()
if is_gpu:
    model = model.cuda()

### 2) Loss func & optimizer

In [45]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## 4. Train

### 실험

#### 환경
- CPU: Intel(R) Core(TM) i7-86565U CPU @ 1.80GHz 1.99Ghz
- GPU: GeForce GTX 1650 Max-Q

#### 10번의 batch 수행 (128 * 10 = 1280개 데이터 처리)
- gpu:  1.3s
- cpu: 24.3s

In [ ]:

for i in range(num_epoch):
    start = time.time()
    i_start = time.time()
        
    for j,[image,label] in enumerate(train_loader):
        optimizer.zero_grad()
        
        if is_gpu:
            x = Variable(image).cuda()
            y_= Variable(label).cuda()
        else:
            x = Variable(image)
            y_= Variable(label)
   
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j % 100 == 0:
            print(f'epoch={i+1:0>2d}, itr={j+1:0>3d}, loss={loss:.4f}, time_elapsed={time.time()-i_start:.2f} secs.') 
            i_start = time.time()
                    
    end = time.time()
    print(f'time elapsed: {end-start:.2f} secs / {(end-start)/60:.2f}mins.')    

epoch=01, itr=001, loss=0.0462, time_elapsed=0.14 secs.
epoch=01, itr=101, loss=0.0063, time_elapsed=13.31 secs.
epoch=01, itr=201, loss=0.0324, time_elapsed=13.31 secs.
epoch=01, itr=301, loss=0.0048, time_elapsed=13.33 secs.
epoch=01, itr=401, loss=0.0085, time_elapsed=13.23 secs.
time elapsed: 62.30 secs / 1.04mins.
epoch=02, itr=001, loss=0.0086, time_elapsed=0.22 secs.
epoch=02, itr=101, loss=0.0671, time_elapsed=13.33 secs.
epoch=02, itr=201, loss=0.0607, time_elapsed=13.34 secs.
epoch=02, itr=301, loss=0.0758, time_elapsed=13.36 secs.
epoch=02, itr=401, loss=0.0083, time_elapsed=13.32 secs.
time elapsed: 62.55 secs / 1.04mins.
epoch=03, itr=001, loss=0.0641, time_elapsed=0.22 secs.
epoch=03, itr=101, loss=0.0010, time_elapsed=13.34 secs.
epoch=03, itr=201, loss=0.0051, time_elapsed=13.33 secs.


## 5. Test

In [29]:
model.eval()

correct = 0
total = 0

for image,label in test_loader:
    if is_gpu:
        x = Variable(image, volatile=True).cuda()
        y_= Variable(label).cuda()
    else:
        x = Variable(image)
        y_= Variable(label)
    
    output = model.forward(x)
    _,output_index = torch.max(output, 1)
    
    total += label.size(0)
    correct += (output_index == y_).sum().float()
    
print(f'Accuracy of test data: {100*correct/total:.2f}%.')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of test data: 98.9300.
